In [12]:
import requests
from bs4 import BeautifulSoup
from selenium import webdriver
from selenium.webdriver.common.desired_capabilitiesimport DesiredCapabilities
import time
import json
from dateutil.parser import parse
import pandas as pd
from datetime import date

In [13]:
url = 'https://apps.ssemc.com/apps/outagemap.aspx' #Snapping Shoals EMC
chrome_driver_path = '/Users/xuanedx1/Desktop/URG/web-scaper/chromedriver'

In [6]:
def get_logs(url):

    desired_capabilities = DesiredCapabilities.CHROME
    desired_capabilities["goog:loggingPrefs"] = {"performance": "ALL"}

    # Create the webdriver object and pass the arguments
    options = webdriver.ChromeOptions()

    # Chrome will start in Headless mode
    # options.add_argument('headless')

    # Ignores any certificate errors if there is any
    options.add_argument("--ignore-certificate-errors")

    # Startup the chrome webdriver with executable path and
    # pass the chrome options and desired capabilities as
    # parameters.
    driver = webdriver.Chrome(executable_path=chrome_driver_path,
                              chrome_options=options,
                              desired_capabilities=desired_capabilities)

    # Send a request to the website and let it load
    driver.get(url)

    # Sleeps for 2 seconds
    time.sleep(5)

    logs_raw = driver.get_log("performance")
    logs = [json.loads(lr["message"])["message"] for lr in logs_raw]
    
    return driver, logs


In [7]:
get_logs(url)

/var/folders/3j/7cy055ys1yz5dcj9cwbxbw8h0000gp/T/ipykernel_1981/2055311750.py:18: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  driver = webdriver.Chrome(executable_path=chrome_driver_path,
/var/folders/3j/7cy055ys1yz5dcj9cwbxbw8h0000gp/T/ipykernel_1981/2055311750.py:18: DeprecationWarning: use options instead of chrome_options
  driver = webdriver.Chrome(executable_path=chrome_driver_path,


(<selenium.webdriver.chrome.webdriver.WebDriver (session="e8df884ecd32003e008815c8a6a773b8")>,
 [{'method': 'Network.loadingFinished',
   'params': {'encodedDataLength': 0,
    'requestId': '517AFD7DE211B9DABDD81632E2FBB902',
    'shouldReportCorbBlocking': False,
    'timestamp': 7232.755453}},
  {'method': 'Page.loadEventFired', 'params': {'timestamp': 7233.003665}},
  {'method': 'Page.frameStoppedLoading',
   'params': {'frameId': '453779B14BD51CAAC7CF7D58784B1DED'}},
  {'method': 'Page.domContentEventFired',
   'params': {'timestamp': 7233.005556}},
  {'method': 'Network.requestWillBeSent',
   'params': {'documentURL': 'https://apps.ssemc.com/apps/outagemap.aspx',
    'frameId': '453779B14BD51CAAC7CF7D58784B1DED',
    'hasUserGesture': False,
    'initiator': {'type': 'other'},
    'loaderId': 'E45CCF3D82DA3BD72BC4546D9FD3F531',
    'redirectHasExtraInfo': False,
    'request': {'headers': {'Upgrade-Insecure-Requests': '1',
      'User-Agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS 

In [8]:
def summary_filter(log_):
    return (
        # is an actual response
        log_["method"] == "Network.responseReceived"
        # and json
        and "json" in log_["params"]["response"]["mimeType"]
        and "summary" in log_["params"]["response"]["url"]
    )

def report_filter(log_):
    return (
        # is an actual response
        log_["method"] == "Network.responseReceived"
        # and json
        and "json" in log_["params"]["response"]["mimeType"]
        and "query" in log_["params"]["response"]["url"]
    )

In [9]:
def get_layout_10_df(url):
    
    driver, logs = get_logs(url)
    
#     summary_response = ""
#     for log in filter(summary_filter, logs):
#         request_id = log["params"]["requestId"]
#         resp_url = log["params"]["response"]["url"]
#         print(f"Caught {resp_url}")
#         summary_response = driver.execute_cdp_cmd("Network.getResponseBody", {"requestId": request_id})['body']
        
#     summary = json.loads(summary_response)
    timestamp = date.today().strftime("%B %d, %Y")
    
    report_responses = []
    for log in filter(report_filter, logs):
        request_id = log["params"]["requestId"]
        resp_url = log["params"]["response"]["url"]
        print(f"Caught {resp_url}")
        report_responses.append(driver.execute_cdp_cmd("Network.getResponseBody", {"requestId": request_id})['body'])
        
    

    rows = []
    for respone in report_responses:
        data = json.loads(respone)
        for area in data['features']:
            try:
                row = {}
                row['timestamp'] = timestamp
                row['county'] = area['attributes']['NAME']
                row['members_affected'] = area['attributes']['o_ConT']
                row['members_served'] = area['attributes']['o_TotalCustomers']

                rows.append(row)
            except:
                pass

    
    layout_10_df = pd.DataFrame(rows, columns = ['timestamp', 'county', 'members_affected', 'members_served'])
        
        
    return layout_10_df.drop_duplicates(keep='last').reset_index(drop = True)
    
    

In [10]:
df = get_layout_10_df(url)

/var/folders/3j/7cy055ys1yz5dcj9cwbxbw8h0000gp/T/ipykernel_1981/2055311750.py:18: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  driver = webdriver.Chrome(executable_path=chrome_driver_path,
/var/folders/3j/7cy055ys1yz5dcj9cwbxbw8h0000gp/T/ipykernel_1981/2055311750.py:18: DeprecationWarning: use options instead of chrome_options
  driver = webdriver.Chrome(executable_path=chrome_driver_path,


In [11]:
df.shape

(0, 4)

In [8]:
df

,timestamp,county,members_affected,members_served
0,"November 03, 2022",DeKalb,0,13930
1,"November 03, 2022",Walton,0,332
2,"November 03, 2022",Jasper,0,398
3,"November 03, 2022",Henry,0,27641
4,"November 03, 2022",Morgan,0,35
5,"November 03, 2022",Rockdale,0,26430
6,"November 03, 2022",Newton,0,30630
7,"November 03, 2022",Butts,0,1301
